## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-27-16-55-57 +0000,bbc,Two Reform UK councillors booted out of party,https://www.bbc.com/news/articles/c867g72nzq4o...
1,2025-10-27-16-53-00 +0000,wsj,Brooklyn has become a symbol of the divide in ...,https://www.wsj.com/us-news/how-mamdanis-brook...
2,2025-10-27-16-50-36 +0000,nyt,Live Updates: Jamaica Prepares for Direct Hit ...,https://www.nytimes.com/live/2025/10/21/weathe...
3,2025-10-27-16-46-18 +0000,nypost,Berkshire Hathaway stock gets rare downgrade —...,https://nypost.com/2025/10/27/business/berkshi...
4,2025-10-27-16-42-00 +0000,wsj,Opinion | Trump Accelerates Our Decline Into M...,https://www.wsj.com/opinion/trump-accelerates-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,46
202,china,20
241,he,13
14,new,12
5,out,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
149,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,139
192,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...,102
193,2025-10-27-04-09-01 +0000,nypost,"US, China talks sketch out rare earths, tariff...",https://nypost.com/2025/10/27/world-news/us-ch...,100
200,2025-10-27-02-01-40 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,98
47,2025-10-27-15-22-03 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
149,139,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
160,70,2025-10-27-09-01-23 +0000,wsj,Stock Market Today: Dow Hits New Record on U.S...,https://www.wsj.com/livecoverage/stock-market-...
59,52,2025-10-27-15-01-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
11,46,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...
239,44,2025-10-26-19-51-18 +0000,nyt,Hurricane Melissa Could Hit as One of Jamaica’...,https://www.nytimes.com/2025/10/26/weather/jam...
21,43,2025-10-27-16-22-44 +0000,nypost,Kathy Hochul oddly drags Buffalo Bills into ‘t...,https://nypost.com/2025/10/27/us-news/kathy-ho...
39,41,2025-10-27-15-42-00 +0000,nypost,Treasury Secretary Bessent says he felt ‘pain’...,https://nypost.com/2025/10/27/business/treasur...
206,37,2025-10-27-00-59-51 +0000,nypost,"US Navy Sea Hawk helicopter, F/A-18F Super Hor...",https://nypost.com/2025/10/26/world-news/us-na...
18,35,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
89,32,2025-10-27-13-44-37 +0000,nyt,Javier Milei’s Party Triumphs in Argentina Mid...,https://www.nytimes.com/2025/10/26/world/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
